In [ ]:
import numpy
import pandas
from scipy.stats import ttest_ind

In [ ]:
#
# Q1
#

# Load only 2013 data
usecols = ['OPEID', 'ICLEVEL', 'UGDS', 'SAT_AVG']
data = pandas.read_csv('CollegeScorecard_Raw_Data/MERGED2013_14_PP.csv',
                       usecols=usecols)
data = data.dropna()
# Filter by 4-year college
data = data[data['ICLEVEL'] == 1]
# Admitted students approximation
data['ADMITTED'] = data['UGDS'] / 4.
# Calculate average SAT score
data['SAT_TOTAL'] = data['SAT_AVG'] * data['ADMITTED']
result = data['SAT_TOTAL'].sum() / data['ADMITTED'].sum()
print('Q1: ', result)

In [ ]:
#
# Q2
#

# Load only 2013 data
usecols = ['ENRL_ORIG_YR2_RT', 'SAT_AVG']
data = pandas.read_csv('CollegeScorecard_Raw_Data/MERGED2013_14_PP.csv',
                       usecols=usecols)
data['ENRL_ORIG_YR2_RT'] = pandas.to_numeric(data['ENRL_ORIG_YR2_RT'], errors='coerce')
data = data.dropna()
# Calculate Pearson correlation
correlations = data.corr(method='pearson')
result = correlations['SAT_AVG']['ENRL_ORIG_YR2_RT']
print('Q2: ', result)

In [ ]:
#
# Q3 & Q4
#

# Load only 2013 data
columns = ['LO_INC_COMP_ORIG_YR4_RT', 'MD_INC_COMP_ORIG_YR4_RT', 'HI_INC_COMP_ORIG_YR4_RT']
usecols = ['ICLEVEL'] + columns
data = pandas.read_csv('CollegeScorecard_Raw_Data/MERGED2013_14_PP.csv',
                       usecols=usecols)
for column in columns:
    data[column] = pandas.to_numeric(data[column], errors='coerce')
data = data.dropna()
# Filter by 4-year college
data = data[data['ICLEVEL'] == 1]
# Mean difference
means = data.mean()
result = means['HI_INC_COMP_ORIG_YR4_RT'] - means['LO_INC_COMP_ORIG_YR4_RT']
print('Q3: ', result)

# Perform t-test
statistic, pvalue = ttest_ind(
    data['LO_INC_COMP_ORIG_YR4_RT'],
    data['HI_INC_COMP_ORIG_YR4_RT'],
    equal_var=True)
result = numpy.log10(pvalue)
print('Q4: ', result)

In [ ]:
#
# Q5
#

# Load only 2013 data
columns = [
    'UGDS_WHITE',
    'UGDS_BLACK',
    'UGDS_HISP',
    'UGDS_ASIAN',
    'UGDS_AIAN',
    'UGDS_NHPI',
    'UGDS_2MOR',
    'UGDS_NRA',
    'UGDS_UNKN',
    'UGDS_WHITENH',
    'UGDS_BLACKNH',
    'UGDS_API',
    'UGDS_AIANOLD',
    'UGDS_HISPOLD',
]
data = pandas.read_csv('CollegeScorecard_Raw_Data/MERGED2013_14_PP.csv',
                       usecols=columns)
# Filter valid entries
data = data[data.max(axis=1) > 0.]
# Minimum difference
diff = data.max(axis=1) - data.min(axis=1)
result = diff.min()
print('Q5: ', result)

In [ ]:
#
# Q6
#

# Load data between 2001 and 2010
columns = ['OPEID', 'UGDS_WOMEN']
data = []
for i in range(1, 11):
    csv = 'CollegeScorecard_Raw_Data/MERGED20%02d_%02d_PP.csv' % (i, i + 1)
    df = pandas.read_csv(csv, usecols=columns, index_col='OPEID', low_memory=False)
    df = df.rename(columns={'UGDS_WOMEN': str(i)})
    df = df[~df.index.duplicated(keep='first')]
    data.append(df)
data = pandas.concat(data, axis=1).dropna()
# Calculate average share of enrollment
result = data.mean().mean()
print('Q6: ', result)

In [ ]:
#
# Q7
#

# Load all data
columns = ['OPEID', 'REGION', 'LOCALE']
data = None
for i in range(2014, 1995, -1):
    csv = 'CollegeScorecard_Raw_Data/MERGED%s_%s_PP.csv' % (i, str(i + 1)[-2:])
    df = pandas.read_csv(csv, usecols=columns, low_memory=False)
    df = df.dropna()
    df = df[~df.index.duplicated(keep='first')]
    if data is None:
        data = df
    else:
        diff = set(df.index) - set(data.index)
        df = df.iloc[pandas.Series(list(diff))]
        data = pandas.concat([data, df])
# Clean data a bit
data['LOCALE'] = data['LOCALE'].astype('int')
data['CITY'] = data['LOCALE'].between(11, 13).astype('int')
data = data.drop(['OPEID', 'LOCALE'], axis=1)
# Calculate max probability
probs = data.groupby('REGION').sum() / data.groupby('REGION').count()
result = probs.max()['CITY']
print('Q7: ', result)